# Semantic Search

In [1]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [5]:
encoder = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B", device='cpu')

In [2]:
from googletrans import Translator

translator = Translator()

In [3]:
clientd = QdrantClient(path="../saves/VScatalogo") # Carrega vectorstore em disco

In [6]:
'''

query_points: metodo para fazer buscas sobre coleção(vectorstore)


Testes preliminares:
    
    Utilizar query em inglês pode trazer resultados mais relevantes, mesmo para buscas em português.
    
    Descrever o papel do modelo na instrução (Instrução:) melhora significativamente a relevância dos resultados.

Futuros caminhos de pesquisa:

    Estudar tasks mais elaboradas

    Talvez utlizar tasks dinamicas, ou seja, descrever o papel do modelo com base na query do usuário.

    Utlizar na vectorestore dados como titulo, nome e descrição para melhorar a relevância dos resultados, algumas descrições não possuem
    nome do nome do catalogo o que leva a uma busca menos precisa.

    Encontrar um meio de filtrar dados do catalogo por Datas, tags, formatos, temas(saúde, educação, segurança pública)...

    Talves dispensar o catalogo e fazer uma vector store unica ou fazer duas vectorstores separadas?

        Caso haja duas vectorstores, como conectar ambas? 
        A vectorestore dentro do catalogo pode ser feita no momento da query?
'''

tasks = ["Você é um motor de busca, devolva dados mais relevantes",
         "Você é um motor de busca, devolva dados mais relevantes com base na busca"]

queries = ['quero dados de criminalidade',
           "Dados do bolsa família",
           "Quero informsções sobre infecções hospitalares",
           "numero de Casamentos"]

query_pt = f"Instrução: {tasks[0]} Query: {queries[3]}"
query_en = await translator.translate(query_pt, src='pt', dest='en')

query = query_en.text

print("Query em português:", query_pt)
print("Query utilizada:", query)

hits = clientd.query_points(
    collection_name="catalogo",
    query=encoder.encode(query).tolist(),
    limit=200,
).points

print(hits)

for hit in hits:
    print("score:", hit.score)
    print("Titulo: ",hit.payload.get('title'))
    print("Nome: ", hit.payload.get('nome'))
    #print("Descrição: ", hit.payload.get('descricao'))
    print('\n')

Query em português: Instrução: Você é um motor de busca, devolva dados mais relevantes Query: numero de Casamentos
Query utilizada: Instruction: You are a search engine, return more relevant data Query: number of Marriages
[ScoredPoint(id=6833, version=0, score=0.4706837276855759, payload={'id': 'f0dd7316-074e-4104-a977-c95a28ad6f46', 'title': 'Nascimentos MEAC', 'nome': 'nascimentos-meac', 'descricao': 'Número total de nascimentos', 'catalogacao': '21/07/2021 18:35:29', 'nomeOrganizacao': 'maternidade-escola-assis-chateaubriand-da-universidade-federal-do-ceara-ch-ufc', 'ultimaAlteracaoMetadados': '29/04/2024 14:35:04', 'ultimaAtualizacaoDados': '08/08/2024 19:27:34', 'isAtualizado': False}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=4005, version=0, score=0.46694576558731793, payload={'id': '60c3513b-59f5-4354-98fb-f82589ec5024', 'title': 'Estatísticas do Registro Civil - RC', 'nome': 'rc-estatisticas-do-registro-civil', 'descricao': 'As Estatísticas do Registro Ci